In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# current cell code gotten from https://www.kaggle.com/stevengolo/classification-and-localization
import xml.etree.ElementTree as etree
ANNOTATION_DIR = '.\VOC2012\Annotations'
def extract_xml_annotation(filename):
    # function gotten from https://www.kaggle.com/stevengolo/classification-and-localization
    """Parse the xml file
    :param filename: str
    """
    z = etree.parse(filename)
    objects = z.findall('./object')
    size = (int(float(z.find('.//width').text)), int(float(z.find('.//height').text)))
    fname = z.find('./filename').text
    dicts = [{obj.find('name').text: [int(float(obj.find('bndbox/xmin').text)),
                                      int(float(obj.find('bndbox/ymin').text)),
                                      int(float(obj.find('bndbox/xmax').text)),
                                      int(float(obj.find('bndbox/ymax').text))]}
             for obj in objects]
    return {'size': size, 'filename': fname, 'objects': dicts}

annotations = []
for filename in sorted(os.listdir(ANNOTATION_DIR)):
    annotation = extract_xml_annotation(os.path.join(ANNOTATION_DIR, filename))
    
    new_objects = []
    for obj in annotation['objects']:
        new_objects.append(obj)
    
    if len(new_objects) == 1:
        annotation['class'] = list(new_objects[0].keys())[0]
        annotation['bbox'] = list(new_objects[0].values())[0]
        annotation.pop('objects')
        annotations.append(annotation)

In [4]:
df = pd.DataFrame(annotations)
# class = 1 means person, class = 0 means not person
def changeToNotPerson(className):
    if (className == 'person'):
        className = '1'
    else:
        className = '0'
    return className
df['class'] = df['class'].apply(changeToNotPerson)
df['class'] = pd.to_numeric(df['class'], downcast='integer')
df.head()

,size,filename,class,bbox
0,"(486, 500)",2007_000027.jpg,1,"[174, 101, 349, 351]"
1,"(500, 375)",2007_000039.jpg,0,"[156, 89, 344, 279]"
2,"(500, 375)",2007_000068.jpg,0,"[27, 45, 266, 375]"
3,"(500, 375)",2007_000123.jpg,0,"[1, 26, 358, 340]"
4,"(500, 332)",2007_000175.jpg,0,"[25, 34, 419, 271]"


In [5]:
# only needs to run once
from tensorflow.keras.applications.efficientnet import preprocess_input

IMAGE_SHAPE = (224, 224)
from tensorflow.keras.preprocessing import image
images = []
for filename in df['filename']:
    print('reading file ' + filename)
    img = image.load_img('./VOC2012/JPEGImages/' + filename, target_size=IMAGE_SHAPE)
    img = image.img_to_array(img)
    img = (img - np.min(img)) / (np.max(img) - np.min(img))
    preprocess_input(img)
    # normalize image
    images.append(img)
images = np.array(images)

reading file 2007_000027.jpg
reading file 2007_000039.jpg
reading file 2007_000068.jpg
reading file 2007_000123.jpg
reading file 2007_000175.jpg
reading file 2007_000187.jpg
reading file 2007_000243.jpg
reading file 2007_000256.jpg
reading file 2007_000272.jpg
reading file 2007_000332.jpg
reading file 2007_000333.jpg
reading file 2007_000491.jpg
reading file 2007_000528.jpg
reading file 2007_000549.jpg
reading file 2007_000636.jpg
reading file 2007_000664.jpg
reading file 2007_000676.jpg
reading file 2007_000720.jpg
reading file 2007_000738.jpg
reading file 2007_000807.jpg
reading file 2007_000837.jpg
reading file 2007_001225.jpg
reading file 2007_001289.jpg
reading file 2007_001321.jpg
reading file 2007_001377.jpg
reading file 2007_001397.jpg
reading file 2007_001423.jpg
reading file 2007_001458.jpg
reading file 2007_001487.jpg
reading file 2007_001568.jpg
reading file 2007_001587.jpg
reading file 2007_001627.jpg
reading file 2007_001667.jpg
reading file 2007_001686.jpg
reading file 2

reading file 2008_000075.jpg
reading file 2008_000076.jpg
reading file 2008_000078.jpg
reading file 2008_000080.jpg
reading file 2008_000084.jpg
reading file 2008_000085.jpg
reading file 2008_000089.jpg
reading file 2008_000095.jpg
reading file 2008_000097.jpg
reading file 2008_000103.jpg
reading file 2008_000105.jpg
reading file 2008_000107.jpg
reading file 2008_000116.jpg
reading file 2008_000120.jpg
reading file 2008_000123.jpg
reading file 2008_000131.jpg
reading file 2008_000148.jpg
reading file 2008_000151.jpg
reading file 2008_000154.jpg
reading file 2008_000162.jpg
reading file 2008_000163.jpg
reading file 2008_000177.jpg
reading file 2008_000181.jpg
reading file 2008_000182.jpg
reading file 2008_000183.jpg
reading file 2008_000187.jpg
reading file 2008_000188.jpg
reading file 2008_000190.jpg
reading file 2008_000192.jpg
reading file 2008_000197.jpg
reading file 2008_000207.jpg
reading file 2008_000213.jpg
reading file 2008_000223.jpg
reading file 2008_000233.jpg
reading file 2

reading file 2008_001679.jpg
reading file 2008_001682.jpg
reading file 2008_001691.jpg
reading file 2008_001692.jpg
reading file 2008_001704.jpg
reading file 2008_001709.jpg
reading file 2008_001712.jpg
reading file 2008_001714.jpg
reading file 2008_001715.jpg
reading file 2008_001716.jpg
reading file 2008_001719.jpg
reading file 2008_001736.jpg
reading file 2008_001741.jpg
reading file 2008_001750.jpg
reading file 2008_001761.jpg
reading file 2008_001769.jpg
reading file 2008_001770.jpg
reading file 2008_001774.jpg
reading file 2008_001783.jpg
reading file 2008_001787.jpg
reading file 2008_001789.jpg
reading file 2008_001796.jpg
reading file 2008_001806.jpg
reading file 2008_001808.jpg
reading file 2008_001810.jpg
reading file 2008_001812.jpg
reading file 2008_001813.jpg
reading file 2008_001821.jpg
reading file 2008_001823.jpg
reading file 2008_001829.jpg
reading file 2008_001832.jpg
reading file 2008_001850.jpg
reading file 2008_001852.jpg
reading file 2008_001871.jpg
reading file 2

reading file 2008_003300.jpg
reading file 2008_003302.jpg
reading file 2008_003303.jpg
reading file 2008_003304.jpg
reading file 2008_003311.jpg
reading file 2008_003326.jpg
reading file 2008_003330.jpg
reading file 2008_003331.jpg
reading file 2008_003343.jpg
reading file 2008_003347.jpg
reading file 2008_003348.jpg
reading file 2008_003359.jpg
reading file 2008_003360.jpg
reading file 2008_003362.jpg
reading file 2008_003369.jpg
reading file 2008_003381.jpg
reading file 2008_003382.jpg
reading file 2008_003384.jpg
reading file 2008_003386.jpg
reading file 2008_003394.jpg
reading file 2008_003409.jpg
reading file 2008_003415.jpg
reading file 2008_003418.jpg
reading file 2008_003420.jpg
reading file 2008_003423.jpg
reading file 2008_003426.jpg
reading file 2008_003433.jpg
reading file 2008_003435.jpg
reading file 2008_003439.jpg
reading file 2008_003443.jpg
reading file 2008_003453.jpg
reading file 2008_003464.jpg
reading file 2008_003480.jpg
reading file 2008_003483.jpg
reading file 2

reading file 2008_004767.jpg
reading file 2008_004768.jpg
reading file 2008_004771.jpg
reading file 2008_004777.jpg
reading file 2008_004794.jpg
reading file 2008_004797.jpg
reading file 2008_004802.jpg
reading file 2008_004804.jpg
reading file 2008_004805.jpg
reading file 2008_004822.jpg
reading file 2008_004825.jpg
reading file 2008_004833.jpg
reading file 2008_004838.jpg
reading file 2008_004845.jpg
reading file 2008_004847.jpg
reading file 2008_004869.jpg
reading file 2008_004873.jpg
reading file 2008_004876.jpg
reading file 2008_004885.jpg
reading file 2008_004893.jpg
reading file 2008_004896.jpg
reading file 2008_004900.jpg
reading file 2008_004904.jpg
reading file 2008_004910.jpg
reading file 2008_004911.jpg
reading file 2008_004914.jpg
reading file 2008_004923.jpg
reading file 2008_004931.jpg
reading file 2008_004933.jpg
reading file 2008_004967.jpg
reading file 2008_004969.jpg
reading file 2008_004970.jpg
reading file 2008_004973.jpg
reading file 2008_004974.jpg
reading file 2

...

reading file 2012_000971.jpg
reading file 2012_000980.jpg
reading file 2012_000984.jpg
reading file 2012_000985.jpg
reading file 2012_000986.jpg
reading file 2012_000991.jpg
reading file 2012_000992.jpg
reading file 2012_000995.jpg
reading file 2012_000996.jpg
reading file 2012_000997.jpg
reading file 2012_000998.jpg
reading file 2012_001000.jpg
reading file 2012_001001.jpg
reading file 2012_001002.jpg
reading file 2012_001003.jpg
reading file 2012_001005.jpg
reading file 2012_001009.jpg
reading file 2012_001010.jpg
reading file 2012_001012.jpg
reading file 2012_001013.jpg
reading file 2012_001014.jpg
reading file 2012_001017.jpg
reading file 2012_001019.jpg
reading file 2012_001023.jpg
reading file 2012_001027.jpg
reading file 2012_001028.jpg
reading file 2012_001032.jpg
reading file 2012_001034.jpg
reading file 2012_001037.jpg
reading file 2012_001038.jpg
reading file 2012_001041.jpg
reading file 2012_001045.jpg
reading file 2012_001046.jpg
reading file 2012_001047.jpg
reading file 2

reading file 2012_001695.jpg
reading file 2012_001696.jpg
reading file 2012_001700.jpg
reading file 2012_001706.jpg
reading file 2012_001707.jpg
reading file 2012_001710.jpg
reading file 2012_001718.jpg
reading file 2012_001724.jpg
reading file 2012_001726.jpg
reading file 2012_001730.jpg
reading file 2012_001732.jpg
reading file 2012_001733.jpg
reading file 2012_001735.jpg
reading file 2012_001744.jpg
reading file 2012_001747.jpg
reading file 2012_001752.jpg
reading file 2012_001754.jpg
reading file 2012_001755.jpg
reading file 2012_001759.jpg
reading file 2012_001760.jpg
reading file 2012_001764.jpg
reading file 2012_001766.jpg
reading file 2012_001767.jpg
reading file 2012_001768.jpg
reading file 2012_001771.jpg
reading file 2012_001775.jpg
reading file 2012_001777.jpg
reading file 2012_001778.jpg
reading file 2012_001784.jpg
reading file 2012_001786.jpg
reading file 2012_001792.jpg
reading file 2012_001794.jpg
reading file 2012_001796.jpg
reading file 2012_001798.jpg
reading file 2

reading file 2012_002424.jpg
reading file 2012_002425.jpg
reading file 2012_002426.jpg
reading file 2012_002428.jpg
reading file 2012_002430.jpg
reading file 2012_002431.jpg
reading file 2012_002434.jpg
reading file 2012_002435.jpg
reading file 2012_002437.jpg
reading file 2012_002440.jpg
reading file 2012_002444.jpg
reading file 2012_002445.jpg
reading file 2012_002447.jpg
reading file 2012_002448.jpg
reading file 2012_002454.jpg
reading file 2012_002459.jpg
reading file 2012_002462.jpg
reading file 2012_002467.jpg
reading file 2012_002468.jpg
reading file 2012_002470.jpg
reading file 2012_002471.jpg
reading file 2012_002472.jpg
reading file 2012_002473.jpg
reading file 2012_002474.jpg
reading file 2012_002478.jpg
reading file 2012_002479.jpg
reading file 2012_002480.jpg
reading file 2012_002482.jpg
reading file 2012_002485.jpg
reading file 2012_002486.jpg
reading file 2012_002492.jpg
reading file 2012_002493.jpg
reading file 2012_002494.jpg
reading file 2012_002497.jpg
reading file 2

reading file 2012_003204.jpg
reading file 2012_003206.jpg
reading file 2012_003210.jpg
reading file 2012_003211.jpg
reading file 2012_003212.jpg
reading file 2012_003213.jpg
reading file 2012_003215.jpg
reading file 2012_003217.jpg
reading file 2012_003221.jpg
reading file 2012_003222.jpg
reading file 2012_003225.jpg
reading file 2012_003226.jpg
reading file 2012_003227.jpg
reading file 2012_003234.jpg
reading file 2012_003238.jpg
reading file 2012_003240.jpg
reading file 2012_003241.jpg
reading file 2012_003251.jpg
reading file 2012_003255.jpg
reading file 2012_003263.jpg
reading file 2012_003264.jpg
reading file 2012_003265.jpg
reading file 2012_003272.jpg
reading file 2012_003273.jpg
reading file 2012_003274.jpg
reading file 2012_003276.jpg
reading file 2012_003277.jpg
reading file 2012_003284.jpg
reading file 2012_003285.jpg
reading file 2012_003287.jpg
reading file 2012_003291.jpg
reading file 2012_003294.jpg
reading file 2012_003295.jpg
reading file 2012_003297.jpg
reading file 2

reading file 2012_003985.jpg
reading file 2012_003986.jpg
reading file 2012_003988.jpg
reading file 2012_003992.jpg
reading file 2012_003995.jpg
reading file 2012_003996.jpg
reading file 2012_004000.jpg
reading file 2012_004001.jpg
reading file 2012_004002.jpg
reading file 2012_004005.jpg
reading file 2012_004006.jpg
reading file 2012_004007.jpg
reading file 2012_004010.jpg
reading file 2012_004011.jpg
reading file 2012_004012.jpg
reading file 2012_004015.jpg
reading file 2012_004018.jpg
reading file 2012_004020.jpg
reading file 2012_004023.jpg
reading file 2012_004034.jpg
reading file 2012_004036.jpg
reading file 2012_004037.jpg
reading file 2012_004038.jpg
reading file 2012_004039.jpg
reading file 2012_004043.jpg
reading file 2012_004044.jpg
reading file 2012_004046.jpg
reading file 2012_004047.jpg
reading file 2012_004048.jpg
reading file 2012_004049.jpg
reading file 2012_004051.jpg
reading file 2012_004053.jpg
reading file 2012_004054.jpg
reading file 2012_004055.jpg
reading file 2

In [6]:
from sklearn.model_selection import train_test_split
# split into training and testing set
X_train, X_test, y_train, y_test = train_test_split(images, np.array(df['class']), test_size=0.15, random_state=42)
# del images

In [240]:
# Initializing and compiling of the model
from tensorflow.keras.applications import EfficientNetB3, MobileNetV2, NASNetMobile, DenseNet169
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input, Activation, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

neurons = 64
lr = 0.01
outputs = 1

base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=IMAGE_SHAPE + (3,), pooling='max')
print(base_model.summary())
base_model.trainable = False

inputs = Input(shape=IMAGE_SHAPE + (3,))
x = base_model(inputs, training=False)
# A Dense classifier with a single unit (binary classification)
x_next = Dense(100, activation='relu')(x)
x_out = Dense(1, activation='sigmoid')(x_next)
model = Model(inputs, x_out)
model.compile(optimizer=Adam(),
                        loss='binary_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "densenet121"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_31 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_34 (ZeroPadding2 (None, 230, 230, 3)  0           input_31[0][0]                   
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_34[0][0]          
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
________________________________________________________________________________________

Model: "functional_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_32 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
densenet121 (Functional)     (None, 1024)              7037504   
_________________________________________________________________
dense_30 (Dense)             (None, 100)               102500    
_________________________________________________________________
dense_31 (Dense)             (None, 1)                 101       
Total params: 7,140,105
Trainable params: 102,601
Non-trainable params: 7,037,504
_________________________________________________________________


In [223]:
ModelCheck = ModelCheckpoint('bestModel_checkpoint2.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto')
hist = model.fit(X_train, y_train, 
              epochs=5, 
              validation_data=(X_test, y_test),
              callbacks=[ModelCheck]
              )

Epoch 1/5
239/239 [==============================] - 22s 94ms/step - loss: 0.3195 - accuracy: 0.8961 - val_loss: 0.1693 - val_accuracy: 0.9442
Epoch 2/5
239/239 [==============================] - 21s 86ms/step - loss: 0.1504 - accuracy: 0.9454 - val_loss: 0.2012 - val_accuracy: 0.9182
Epoch 3/5
239/239 [==============================] - 20s 85ms/step - loss: 0.1365 - accuracy: 0.9461 - val_loss: 0.2373 - val_accuracy: 0.9011
Epoch 4/5
239/239 [==============================] - 21s 86ms/step - loss: 0.1214 - accuracy: 0.9546 - val_loss: 0.1725 - val_accuracy: 0.9286
Epoch 5/5
239/239 [==============================] - 21s 88ms/step - loss: 0.0967 - accuracy: 0.9614 - val_loss: 0.1869 - val_accuracy: 0.9450


In [1]:
# loads best model from the training phase
from tensorflow.keras.models import load_model
model = load_model('bestModel_checkpoint2.h5')

In [2]:
import matplotlib.pyplot as plt
f, ax = plt.subplots()
ax.plot([None] + hist.history['accuracy'], '--')
ax.plot([None] + hist.history['val_accuracy'])
# Plot legend and use the best location automatically: loc = 0.
ax.legend(['Train acc', 'Validation acc'], loc = 0)
ax.set_title('Training/Validation acc per Epoch')
ax.set_xlabel('Epoch')
ax.set_ylabel('acc')

NameError: name 'hist' is not defined

In [8]:
# testing with different images from the webcam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import decode_predictions
import numpy as np
IMAGE_SHAPE=(224,224)
img = image.load_img('WIN_20201015_13_50_45_Pro.jpg', target_size=IMAGE_SHAPE)
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = (img - np.min(img)) / (np.max(img) - np.min(img))
# img = preprocess_input(img)

In [11]:
%%time
prediction = model.predict(img)

Wall time: 43 ms


In [12]:
prediction

array([[0.7145978]], dtype=float32)

In [ ]:
# converting Keras model into a Tensorflow.js model
from tensorflowjs.converters import save_keras_model
save_keras_model(model, './EN3_PersonNoPerson_classifier_TfJS')